In [7]:
import emcee

import matplotlib.pyplot as plt
import numpy as np
import os, sys, shutil
import pandas as pd
import sklearn.metrics as slm

import corner
from scipy.interpolate import interp1d

In [8]:
import sys
import numpy as np

pkdir = "/pscratch/sd/s/sbrisin/boss_cib_cmass/cibcmass/hmvec"
sys.path.insert(0, pkdir)

        # Import hmvec as hm
from hmvec import hmvec as hm
ells = np.arange(3000)
cib_freq = 545 * 1e9


zs = np.linspace(.4, .5, 5)
ms = np.geomspace(1e10, 1e17, 50)
ks = np.geomspace(1e-4, 100, 100)

hcos = hm.HaloModel(zs, ks, ms=ms)
hcos.set_cibParams('vierro')

        # a and b are the z range
hcos.cib_params['alpha'] = 0.2
hcos.cib_params['beta'] = 1.6
hcos.cib_params['gamma'] = 1.7  # not in Viero, so using Planck13
hcos.cib_params['delta'] = 2.4
hcos.cib_params['Td_o'] = 20.7
logmeff = hcos.cib_params['logM_eff']
hcos.cib_params['var'] = 0.3
l0 = hcos.cib_params['L_o']
hcos.add_hod(name="CMASS", mthresh=10**12 + zs*0.)
ells = np.arange(300,1000)

In [69]:
def cig_mod(theta):
    x,y = theta
    hcos.cib_params['L_o'] = x
    hcos.cib_params['logM_eff'] = y
            # Power spectra for CIB x galaxies
    PgI_1h = hcos.get_power_1halo('CMASS', 'cib', nu_obs=np.array([cib_freq]))
    PgI_2h = hcos.get_power_2halo('CMASS', 'cib', nu_obs=np.array([cib_freq]))

    Cl_gI_1h = hcos.C_gI(ells, hcos.zs, hcos.ks, PgI_1h, gzs=zs, gdndz=np.ones_like(zs))
    Cl_gI_2h = hcos.C_gI(ells, hcos.zs, hcos.ks, PgI_2h, gzs=zs, gdndz=np.ones_like(zs))
    
    # need to end up with one vector thats c_gi for each l bin, c_gg for each l bin
    tot = Cl_gI_1h + Cl_gI_2h
    
    df = pd.DataFrame({'ells': ells, 'tot': tot})
    b1 = tot[ells == 370]
    b2 = tot[ells == 510]
    b3 = tot[ells == 650] 
    b4 = tot[ells == 790.0]
    b5 = tot[ells == 930]
    b = np.array([b1,b2,b3,b4,b5])
    return b

In [63]:
def cgg_mod(theta):
    x,y = theta
    hcos.cib_params['L_o'] = x
    hcos.cib_params['logM_eff'] = y
            # Power spectra for CIB x galaxies
    Pgg_1h = hcos.get_power_1halo('CMASS', 'CMASS', nu_obs=np.array([cib_freq]))
    Pgg_2h = hcos.get_power_2halo('CMASS', 'CMASS', nu_obs=np.array([cib_freq]))

    Cl_gg_1h = hcos.C_gg(ells, hcos.zs, hcos.ks, Pgg_1h, gzs=zs, gdndz=np.ones_like(zs))
    Cl_gg_2h = hcos.C_gg(ells, hcos.zs, hcos.ks, Pgg_2h, gzs=zs, gdndz=np.ones_like(zs))
    
    # need to end up with one vector thats c_gi for each l bin, c_gg for each l bin
    tot = Cl_gg_1h + Cl_gg_2h
    
    b1 = tot[ells == 70]
    b2 = tot[ells == 210]
    b3 = tot[ells == 350] 
    b4 = tot[ells == 490.0]
    b5 = tot[ells == 630]
    b = np.vectorize([b1,b2,b3,b4,b5])
    return b

def cii_mod(theta):
    x,y = theta
    zs = np.linspace(.01,5,20)
    ms = np.geomspace(1e10,1e17,200)
    ks = np.geomspace(1e-4,100,1001)
    hcos = hm.HaloModel(zs,ks,ms=ms)
    hcos.set_cibParams('vierro')
    hcos.cib_params['L_o'] = x
    hcos.cib_params['logM_eff'] = y
            # Power spectra for CIB x galaxies
    Pii_1h = hcos.get_power_1halo('cib', 'cib', nu_obs=np.array([cib_freq]))
    Pii_2h = hcos.get_power_2halo('cib', 'cib', nu_obs=np.array([cib_freq]))

    Cl_ii_1h = hcos.C_ii(ells, hcos.zs, hcos.ks, Pii_1h)
    Cl_ii_2h = hcos.C_ii(ells, hcos.zs, hcos.ks, Pii_2h)
    
    # need to end up with one vector thats c_gi for each l bin, c_gg for each l bin
    tot = Cl_ii_1h + Cl_ii_2h
    
    b1 = tot[ells == 70]
    b2 = tot[ells == 210]
    b3 = tot[ells == 350] 
    b4 = tot[ells == 490.0]
    b5 = tot[ells == 630]
    b = np.vectorize([b1,b2,b3,b4,b5])
    return b

In [64]:
dat = np.array([1.506013e-08, 
1.593685e-08,
1.071126e-08, 
1.097310e-08, 
5.175340e-09]) # this is your "data"
cov = np.array([[[ 2.04832555e-07, -1.19540773e-15],
       [-1.19540773e-15,  1.89627266e-15]],[[1.44795000e-07, 2.04875041e-13],
       [2.04875041e-13, 8.77247382e-16]],[[1.11634841e-07, 3.00682665e-16],
       [3.00682665e-16, 3.96679922e-16]],[[8.71831675e-08, 3.30496239e-15],
       [3.30496239e-15, 2.41161963e-16]],[[7.33325061e-08, 4.68626176e-16],
       [4.68626176e-16, 1.54683514e-16]]])

# this is your covariance matrix
cinv = np.linalg.inv(cov) # inverse covariance matrix

In [70]:
def model(theta, dat = dat):
    x,y =  theta
    return cig_mod(theta)
def lnlike(theta,dat,cov,cinv):
    gauss_like = lambda thy: -0.5*np.dot(np.dot(thy-dat,cinv),thy-dat)
    return gauss_like
def lnprior(theta):
    x,y = theta
    return 0.0
def lnprob(theta):
    lp = lnprior(theta)
    if not lp == -np.inf:
        return -np.inf
    return lp + lnlike(theta)

In [71]:
def main(p0,nwalkers,niter,ndim,lnprob,data):
    sampler =  emcee.EnsembleSampler(nwalkers,ndim, lnprob, args = data)
    print('running baybee..')
    p0, _, _ = sampler.run_mcmc(p0,100)
    sampler.reset()
    
    print( 'running prod')
    pos, prob, state =  sampler.run_mcmc(p0,niter)
    
    return sampler, pos, prob, state

In [72]:
data = (dat,cov,cinv)
nwalkers =  100
niter = 10000
# set n walkers
# set niter
initial = np.array([2e-7,12.3])
ndim = 2
p0 = [np.array(initial) + np.random.randn(ndim) for i in range(nwalkers)]

In [73]:
sampler, pos, prob, state = main(p0,nwalkers,niter,ndim,lnlike,data)

running baybee..


TypeError: float() argument must be a string or a number, not 'function'

In [ ]:
samples = sampler.flatchain

In [ ]:
labels = ['test', 'test2']
fig = corner.corner(samples,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])